<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#EC2-Instance-Stop" data-toc-modified-id="EC2-Instance-Stop-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EC2 Instance Stop</a></span></li><li><span><a href="#EC2-Snapshots-Deletion" data-toc-modified-id="EC2-Snapshots-Deletion-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EC2 Snapshots Deletion</a></span></li></ul></div>

# EC2 Instance Stop

In [1]:
'''Load Environment Variables'''
%env region=ap-south-1

env: region=ap-south-1


In [2]:
import logging

In [16]:
#!/usr/bin/python
'''
References:
1) JSON Role Policy : lambda_deployer_role : https://marcelog.github.io/articles/aws_lambda_start_stop_ec2_instance.html
2) Logging Hadler   : Re-Invent            : https://gist.github.com/niranjv/fb95e716151642e8ca553b0e38dd152e
3) Lambda Issue VPC : Can't connect to ISP : https://docs.aws.amazon.com/lambda/latest/dg/configuration-vpc.html#vpc-internet
'''
import os
import sys
import time
import json
import boto3
import datetime

'''importing custom modules'''
'''Jupyter Notebook - Log file'''
# sys.path.append(os.path.dirname(os.path.realpath(__file__)) + '/pySetenv/variables/' )
# sys.path.append(os.path.dirname(os.path.realpath(__file__)) + '/pySetenv/packages/'  )

# import lambda_logger
# logger = lambda_logger.setup_lambda_logger()

'''Jupyter Notebook - Log file'''
# sys.path.append(os.path.dirname('C:\\Users\\Vignesh_Palanivel\\Google Drive\\Jupyter Notebooks\\pySetenv\\'))
# print (sys.path)

# import logger
# execLogger  = 'rti-upload-download' + time.strftime('-%Y-%m-%d-%Hh-%Mm-%Ss-%Z') + '.log'
# logger      = logger.setupLogger('Stop Instances Logger', execLogger)
'''Jupyter Notebook - Stream'''
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='[%(asctime)s] - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S %Z')

today_date      = datetime.datetime.now(datetime.timezone.utc).astimezone(gettz("Asia/Kolkata"))
logger.info('Today"s Date                : {}'.format(today_date))

def json_serial(dt_object):
    '''
    Function to convert the datatime object to string object(ios format)
    
    Arguments:
        dt_object (dataetime) : Datetime object 
    Returns:
        iso format datetime(String)
    '''
    if isinstance(dt_object, (datetime.date, datetime.datetime)):
        return dt_object.isoformat()

def describe_instance(ec2_client, filter):
    '''
    Function to Describe the EC2 instance and Filter instances based on the filter condition passed
    
    Arguments:
        ec2_client (ec2 client) : EC2 Client object
        filter     (List)       : List of dictionary with Filter values
    Returns:
        List of Instance IDs
    '''
    describe_instances    = ec2_client.describe_instances(Filters= filter)
    instance_id_list      = [instance['InstanceId'] for reservations in describe_instances['Reservations'] for instance in reservations['Instances']]
    # logger.info(instance_id_list)
    return instance_id_list

''' Main Function'''
def stop_instance(event, lambda_context):
    '''
    Function to Stop the Ec2 Instances
    
    Arguments:
        event          (None) : No Events Required
        lambda_context (None) : No Context Required
    Actions:
        Stopping the instances received from describe_instance function
    
    Calling Function:
        describe_instance (To get the list of filtered instances)
        json_serial       (to process output from describe_instance)
    '''
    
    '''Reading Environment Variables'''
    # logger.info('ENVIRONMENT VARIABLES')
    # logger.info(os.environ)
    region = os.environ['region']
    logger.info('ENV - Region                  : {}'.format(region))

    '''Initializing all necessory variables'''
    print('\n')
    logger.info('Initializing boto3 client     :  EC2')
    ec2_client                = boto3.client('ec2', region_name=region)
    
    ec2_stop_instances_waiter = ec2_client.get_waiter('instance_stopped')
    filter_running_instance   = [{ 'Name'   : 'instance-state-name', 'Values' : ['pending','running']},
                                 { 'Name'   : 'tag:Scheduled',       'Values' : ['True']}]
    filter_stopped_instance   = [{ 'Name'   : 'instance-state-name', 'Values' : ['stopped']}]
    
    logger.info('Fetching Running EC2 instance Details')
    instance_id_list = describe_instance(ec2_client, filter_running_instance)
    
    if instance_id_list:
        try: 
            logger.info('Stopping Instance(s)          : {} ...'.format(instance_id_list))
            stop_trigger   = ec2_client.stop_instances(InstanceIds = instance_id_list)
            if stop_trigger['ResponseMetadata']['HTTPStatusCode'] == 200:
                logger.info(json.dumps(stop_trigger['StoppingInstances'], indent=4, separators=(',', ': '), default=json_serial))
            ec2_stop_instances_waiter.wait(Filters=filter_stopped_instance)
            logger.info('Stopped Instance(s)           : {}'.format(instance_id_list))
            logger.info('# Lambda Function status      : Completed #\n\n')
        except Exception as Error:
            logger.error('%s' %Error)
            logger.error('Failure in Stopping Instance : {} !!!'.format(instance_id_list))
            logger.error('# Lambda Function status     : Failed #\n\n')
    else:
        logger.warning('No instance is in Running State')
        logger.warning('# Lambda Function status   : None #\n\n')

[2020-05-30 11:30:06 India Standard Time] - INFO - Today"s Date                : 2020-05-30 11:30:06.312092+05:30


In [8]:
'''Instances in Running state'''
stop_instance(None, None)

[2020-05-23 17:33:54 India Standard Time] - INFO - Initializing boto3 client -  EC2
[2020-05-23 17:33:54 India Standard Time] - INFO - Fetching Running EC2 instance Details


[2020-05-23 17:33:55 India Standard Time] - INFO - Stopping Instance(s) : ['i-09adf316ecee00468', 'i-077007a2fb8930489'] ...
[2020-05-23 17:33:55 India Standard Time] - INFO - [
    {
        "CurrentState": {
            "Code": 64,
            "Name": "stopping"
        },
        "InstanceId": "i-077007a2fb8930489",
        "PreviousState": {
            "Code": 16,
            "Name": "running"
        }
    },
    {
        "CurrentState": {
            "Code": 64,
            "Name": "stopping"
        },
        "InstanceId": "i-09adf316ecee00468",
        "PreviousState": {
            "Code": 16,
            "Name": "running"
        }
    }
]
[2020-05-23 17:34:10 India Standard Time] - INFO - Stopped Instance(s) : ['i-09adf316ecee00468', 'i-077007a2fb8930489']
[2020-05-23 17:34:10 India Standard Time] - INFO - # Lambda Function status : Completed #




In [17]:
'''Instances in stopped state'''
stop_instance(None, None)

[2020-05-30 11:30:11 India Standard Time] - INFO - ENV - Region                  : ap-south-1
[2020-05-30 11:30:11 India Standard Time] - INFO - Initializing boto3 client     :  EC2
[2020-05-30 11:30:11 India Standard Time] - INFO - Fetching Running EC2 instance Details


[2020-05-30 11:30:12 India Standard Time] - WARNING - No instance is in Running State
[2020-05-30 11:30:12 India Standard Time] - WARNING - # Lambda Function status   : None #


[2020-05-30 11:30:12 India Standard Time] - INFO - # End of EC2 Stop Activity #


# EC2 Snapshots Deletion

In [1]:
'''Load Environment Variables'''
%env region         = ap-south-1
%env tolerated_days = 0

env: region=ap-south-1
env: tolerated_days=0


In [2]:
import logging

In [7]:
#!/usr/bin/python
'''
References:
1) JSON Role Policy : lambda_deployer_role : https://marcelog.github.io/articles/aws_lambda_start_stop_ec2_instance.html
2) Logging Hadler   : Re-Invent            : https://gist.github.com/niranjv/fb95e716151642e8ca553b0e38dd152e
3) Lambda Issue VPC : Can't connect to ISP : https://docs.aws.amazon.com/lambda/latest/dg/configuration-vpc.html#vpc-internet
4) Date Time Ex     : Examples             : https://www.techatbloomberg.com/blog/work-dates-time-python/
5) Snapshot Ex      : Example Program      : https://gist.github.com/nirbhabbarat/047059f87138e04b2fe83bf678161fff
6) Snapshot Ex      : Example Program      : https://gist.github.com/kjoconnor/7344485
6) Snapshot Ex      : Example Program      : https://stackoverflow.com/questions/48124269/python-boto3-program-to-delete-old-snapshots-in-aws
'''
import os
import sys
import time
import json
import boto3
import datetime
import pytz
from dateutil.tz import tzutc
from dateutil.tz import gettz

'''importing custom modules'''
'''Setting Up Lambda logger'''
# sys.path.append(os.path.dirname(os.path.realpath(__file__)) + '/pySetenv/variables/' )
# sys.path.append(os.path.dirname(os.path.realpath(__file__)) + '/pySetenv/packages/'  )

# import lambda_logger
# logger = lambda_logger.setup_lambda_logger()

'''Jupyter Notebook - Log file'''
# sys.path.append(os.path.dirname('C:\\Users\\Vignesh_Palanivel\\Google Drive\\Jupyter Notebooks\\pySetenv\\'))
# print (sys.path)

# import logger
# execLogger  = 'rti-upload-download' + time.strftime('-%Y-%m-%d-%Hh-%Mm-%Ss-%Z') + '.log'
# logger      = logger.setupLogger('Stop Instances Logger', execLogger)
'''Jupyter Notebook - Stream'''
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='[%(asctime)s] - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S %Z')

'''Defining Global Variables'''
ss_in_24hrs     = []
ss_in_5days     = []
ss_recent       = []
ss_id_deleted   = []
today_date      = datetime.datetime.now(datetime.timezone.utc).astimezone(gettz("Asia/Kolkata"))
logger.info('Today"s Date                : {}'.format(today_date))

def json_serial(dt_object):
    '''
    Function to convert the datatime object to string object(ios format)
    
    Arguments:
        dt_object (dataetime) : Datetime object 
    Returns:
        iso format datetime(String)
    '''
    if isinstance(dt_object, (datetime.date, datetime.datetime)):
        return dt_object.isoformat()

def future_calculations(snapshot, tolerated_days):
    '''
    Function to calculate the future snapshot deletion dates and intimate accordingly
    
    Arguments:
        
    Returns:
        
    '''
    today_date      = today_date.date()
    ss_created_date = snapshot['StartTime'].astimezone(gettz("Asia/Kolkata")).date()
    
    snapshot_id     = snapshot['SnapshotId']
    delete          = False
    
    days_old        = abs((ss_created_date - today_date).days)
    
    if days_old == tolerated_days:
        delete = True
    elif days_old == (tolerated_days - 1):
        ss_in_24hrs.append(snapshot_id)
    elif days_old >= (tolerated_days - 5) and days_old < (tolerated_days - 1):
        ss_in_5days.append(snapshot_id)
    else:
        ss_recent.append(snapshot_id)
    
    return delete
    
''' Main Function'''
def snapshots_cleanup(event, lambda_context):
    '''
    Function to cleanup the Ec2 snapshots older than X Days
    
    Arguments:
        event          (None) : No Events Required
        lambda_context (None) : No Context Required
    Returns:
        Deleting the snapshots ids reveived from he function describe_snapshots
    
    Calling Function:
        describe_snapshots (To get the list of filtered snapshots)
        json_serial       (to process output from describe_snapshots)
    '''
    
    '''Reading Environment Variables'''
    # logger.info('ENVIRONMENT VARIABLES')
    # logger.info(os.environ)
    region         = os.environ['region']
    tolerated_days = int(os.environ['tolerated_days'])
    logger.info('ENV - Region                : {}'.format(region))
    logger.info('ENV - Tolerated Days        : {}'.format(tolerated_days))
    

    '''Initializing all necessory variables'''
    print('\n')
    logger.info('Initializing boto3 client   :  EC2')
    logger.info('Initializing boto3 client   :  STS')
    ec2_client      = boto3.client('ec2', region_name=region)
    sts_client      = boto3.client('sts', region_name=region)
    
    logger.info('Describing all the EC2 Snapshots')
    ss_paginator    = ec2_client.get_paginator('describe_snapshots')
    ss_filters      = [{ 'Name': 'tag:Team', 'Values': ['terraform-services-india']}]
    account_id      = sts_client.get_caller_identity()["Account"]
    
    ss_page_iterator = ss_paginator.paginate(
        Filters          = ss_filters,
        OwnerIds         = [account_id], #'self'
        PaginationConfig = {'PageSize': 200}
    )
    
    logger.info('Calculating Snapshots age  ...')
    # print(ss_page_iterator)
    for page in ss_page_iterator:
        # print(json.dumps(page, indent=4, separators=(',', ': '), default=json_serial))
        for snapshot in page['Snapshots']:
            # print(json.dumps(snapshot, indent=4, separators=(',', ': '), default=json_serial))
            # ss_created_date = dateutil.parser.parse(str(snapshot['StartTime'].date())))
            
            action      = future_calculations(snapshot, tolerated_days)
            
            if action:
                snapshot_id = snapshot['SnapshotId']
                try:
                    dry_run = True
                    ec2_client.delete_snapshot(SnapshotId=snapshot_id, DryRun=dry_run)
                except Exception as Error_Msg:
                    dry_run = False
                    if 'DryRunOperation' in str(Error_Msg):
                        result = ec2_client.delete_snapshot(SnapshotId=snapshot_id, DryRun=dry_run)
                        if result['ResponseMetadata']['HTTPStatusCode'] == 200:
                            logger.info('Successfully Deleted SS    : {}'.format(snapshot_id))
                    elif 'InvalidSnapshot.InUse' in str(Error_Msg):
                        logger.warning('Snapshot In-Use skipping   : {}'.format(snapshot_id))
                        continue
                    else:
                        logger.error('Couldn"t Delete Snapshot   : {}'.format(snapshot_id))
                finally:
                    ss_id_deleted.append(snapshot_id)
    
    logger.info('Snapshot Report as Follows...\n')
    if ss_id_deleted:
        logger.info('Snapshots Deleted Now      : {}'.format(len(ss_id_deleted)))
    else:
        logger.info('Snapshots Breached         : None')
    
    if ss_in_24hrs:
        logger.info('Snapshots in Tolerated     : {}'.format(len(ss_in_24hrs)))
    else:
        logger.info('Snapshots in Tolerated     : None')
    
    if ss_in_5days:
        logger.info('Snapshots in Queue         : {}'.format(len(ss_in_5days)))
    else:
        logger.info('Snapshots in Queue         : None')
        
    if ss_recent:
        logger.info('Snapshots in Recent        : {}'.format(len(ss_recent)))
    else:
        logger.info('Snapshots in Recent        : None\n')
    
    logger.info('# End of EC2 Snapshots Clean-Up Activity #')

[2020-05-30 11:17:37 India Standard Time] - INFO - Today"s Date                : 2020-05-30 11:17:37.743864+05:30


In [104]:
'''Snapshot in Recent'''
snapshots_cleanup(None, None)

[2020-05-26 21:47:23 India Standard Time] - INFO - ENV - Region                : ap-south-1
[2020-05-26 21:47:23 India Standard Time] - INFO - ENV - Tolerated Days        : 15
[2020-05-26 21:47:23 India Standard Time] - INFO - Initializing boto3 client   :  EC2
[2020-05-26 21:47:23 India Standard Time] - INFO - Initializing boto3 client   :  STS
[2020-05-26 21:47:23 India Standard Time] - INFO - Describing all the EC2 Snapshots


[2020-05-26 21:47:24 India Standard Time] - INFO - Calculating Snapshots age  ...
[2020-05-26 21:47:24 India Standard Time] - INFO - Completed calculations   !!!

[2020-05-26 21:47:24 India Standard Time] - INFO - Snapshots Breached         : None
[2020-05-26 21:47:24 India Standard Time] - INFO - Snapshots in Tolerated     : None
[2020-05-26 21:47:24 India Standard Time] - INFO - Snapshots in Queue         : None
[2020-05-26 21:47:24 India Standard Time] - INFO - Snapshots in Recent        : 2
[2020-05-26 21:47:24 India Standard Time] - INFO - # End of EC2 Snapshots Clean-Up Activity


In [112]:
'''Snapshot in Queue'''
snapshots_cleanup(None, None)

[2020-05-26 22:18:46 India Standard Time] - INFO - ENV - Region                : ap-south-1
[2020-05-26 22:18:46 India Standard Time] - INFO - ENV - Tolerated Days        : 2
[2020-05-26 22:18:46 India Standard Time] - INFO - Initializing boto3 client   :  EC2
[2020-05-26 22:18:46 India Standard Time] - INFO - Initializing boto3 client   :  STS
[2020-05-26 22:18:46 India Standard Time] - INFO - Describing all the EC2 Snapshots


[2020-05-26 22:18:49 India Standard Time] - INFO - Calculating Snapshots age  ...
[2020-05-26 22:18:49 India Standard Time] - INFO - Completed calculations   !!!

[2020-05-26 22:18:49 India Standard Time] - INFO - Snapshots Breached         : None
[2020-05-26 22:18:49 India Standard Time] - INFO - Snapshots in Tolerated     : None
[2020-05-26 22:18:49 India Standard Time] - INFO - Snapshots in Queue         : 2
[2020-05-26 22:18:49 India Standard Time] - INFO - Snapshots in Recent        : None

[2020-05-26 22:18:49 India Standard Time] - INFO - # End of EC2 Snapshots Clean-Up Activity


In [108]:
'''Snapshot in Tolerated'''
snapshots_cleanup(None, None)

[2020-05-26 21:47:52 India Standard Time] - INFO - ENV - Region                : ap-south-1
[2020-05-26 21:47:52 India Standard Time] - INFO - ENV - Tolerated Days        : 1
[2020-05-26 21:47:52 India Standard Time] - INFO - Initializing boto3 client   :  EC2
[2020-05-26 21:47:52 India Standard Time] - INFO - Initializing boto3 client   :  STS
[2020-05-26 21:47:52 India Standard Time] - INFO - Describing all the EC2 Snapshots


[2020-05-26 21:47:53 India Standard Time] - INFO - Calculating Snapshots age  ...
[2020-05-26 21:47:56 India Standard Time] - INFO - Completed calculations   !!!

[2020-05-26 21:47:56 India Standard Time] - INFO - Snapshots Breached         : None
[2020-05-26 21:47:56 India Standard Time] - INFO - Snapshots Deleted in 24hrs : 2
[2020-05-26 21:47:56 India Standard Time] - INFO - Snapshots in Queue         : None
[2020-05-26 21:47:56 India Standard Time] - INFO - Snapshots in Recent        : None

[2020-05-26 21:47:56 India Standard Time] - INFO - # End of EC2 Snapshots Clean-Up Activity


In [6]:
'''Snapshot in Breached'''
snapshots_cleanup(None, None)

[2020-05-26 22:39:05 India Standard Time] - INFO - ENV - Region                : ap-south-1
[2020-05-26 22:39:05 India Standard Time] - INFO - ENV - Tolerated Days        : 0
[2020-05-26 22:39:05 India Standard Time] - INFO - Initializing boto3 client   :  EC2
[2020-05-26 22:39:05 India Standard Time] - INFO - Initializing boto3 client   :  STS
[2020-05-26 22:39:05 India Standard Time] - INFO - Found credentials in shared credentials file: ~/.aws/credentials


[2020-05-26 22:39:05 India Standard Time] - INFO - Describing all the EC2 Snapshots
[2020-05-26 22:39:09 India Standard Time] - INFO - Calculating Snapshots age  ...
[2020-05-26 22:39:11 India Standard Time] - INFO - Successfully Deleted SS    : snap-06944cffc53d0e1de
[2020-05-26 22:39:12 India Standard Time] - INFO - Successfully Deleted SS    : snap-05536c5dd1a6a6d3a
[2020-05-26 22:39:12 India Standard Time] - INFO - Snapshot Report as Follows..

[2020-05-26 22:39:12 India Standard Time] - INFO - Snapshots Deleted Now      : 2
[2020-05-26 22:39:12 India Standard Time] - INFO - Snapshots in Tolerated     : None
[2020-05-26 22:39:12 India Standard Time] - INFO - Snapshots in Queue         : None
[2020-05-26 22:39:12 India Standard Time] - INFO - Snapshots in Recent        : None

[2020-05-26 22:39:12 India Standard Time] - INFO - # End of EC2 Snapshots Clean-Up Activity
